In [2]:
pip install "pyiceberg[s3fs,glue]"

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.1/298.1 KB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 KB 10.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 12.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 15.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 15.3 MB/s eta 0:00:0000

In [18]:
pip install "pyiceberg[pyarrow]"

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 12.6 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [40]:
# Iceberg- catalog; Load the catalog and list out the databases already existing in Glue:
from pyiceberg.catalog import load_catalog
catalog = load_catalog("glue", **{"type": "glue"})
catalog.list_namespaces()

[('database',), ('db',), ('db2',), ('db3',), ('test',)]

In [16]:
# Listing out the existing Iceberg tables in the test catalog:
catalog.list_tables("test")

[('test', 'sales'),
 ('test', 'salesnew'),
 ('test', 'test'),
 ('test', 'nyc_worker_coops')]

In [25]:
# This returns a Table that represents an Iceberg table:
ice_table = catalog.load_table("test.salesnew")

In [42]:
# to query an Ice table, we need to do a table scan using this method:
scan = ice_table.scan()

In [45]:
# For analysis, we convert our table scan into a PyArrow object:
scan = scan.to_arrow()

In [46]:
print(scan)

pyarrow.Table
id: string
name: string
product: string
price: string
date: string
----
id: [["1","2","3","4","5",...,"496","497","498","499","500"]]
name: [["Selinda Rheaume","Wynnie Gozzard","Patten Whitter","Hulda Eslie","Chrystal Haggie",...,"Winny McGlone","Hadleigh Ellinor","Kimberlee Hancill","Anet Scaife","Bary Reap"]]
product: [["Wine - Prosecco Valdobiaddene","Wine - Red, Wolf Blass, Yellow","Crackers - Soda / Saltins","Roe - White Fish","Wine - Delicato Merlot",...,"Pasta - Fusili Tri - Coloured","Oats Large Flake","Soup - Knorr, Country Bean","Soup - French Onion","Soup - Tomato Mush. Florentine"]]
price: [["10.31","20.05","39.75","37.10","35.82",...,"67.78","58.19","81.64","98.51","92.64"]]
date: [["10/20/2022","10/20/2022","10/20/2022","10/20/2022","10/20/2022",...,"10/20/2022","10/20/2022","10/20/2022","10/20/2022","10/20/2022"]]


In [47]:
# Convert it into a dataframe
df = scan.to_pandas()

In [48]:
df

,id,name,product,price,date
0,1,Selinda Rheaume,Wine - Prosecco Valdobiaddene,10.31,10/20/2022
1,2,Wynnie Gozzard,"Wine - Red, Wolf Blass, Yellow",20.05,10/20/2022
2,3,Patten Whitter,Crackers - Soda / Saltins,39.75,10/20/2022
3,4,Hulda Eslie,Roe - White Fish,37.10,10/20/2022
4,5,Chrystal Haggie,Wine - Delicato Merlot,35.82,10/20/2022
...,...,...,...,...,...
495,496,Winny McGlone,Pasta - Fusili Tri - Coloured,67.78,10/20/2022
496,497,Hadleigh Ellinor,Oats Large Flake,58.19,10/20/2022
497,498,Kimberlee Hancill,"Soup - Knorr, Country Bean",81.64,10/20/2022
498,499,Anet Scaife,Soup - French Onion,98.51,10/20/2022


In [49]:
print(ice_table.metadata.snapshots)

[Snapshot(snapshot_id=3393406712675561257, parent_snapshot_id=None, sequence_number=None, timestamp_ms=1673476027663, manifest_list='s3://dm-iceberg/test.db/salesnew/metadata/snap-3393406712675561257-1-76b0943d-ceb3-454e-ae42-14ff9acb156b.avro', summary=Summary(Operation.APPEND, **{'spark.app.id': 'local-1673475113466', 'added-data-files': '1', 'added-records': '500', 'added-files-size': '14771', 'changed-partition-count': '1', 'total-records': '500', 'total-files-size': '14771', 'total-data-files': '1', 'total-delete-files': '0', 'total-position-deletes': '0', 'total-equality-deletes': '0'}), schema_id=0)]
